In [1]:
import tensorflow as tf
from tensorflow import keras
from glob import glob
import os
import numpy as np
import random

2024-01-04 21:10:31.008438: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-04 21:10:31.163712: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-04 21:10:31.847620: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-01-04 21:10:31.847688: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

# creating classifier

In [2]:
autoencoder_model = tf.keras.models.load_model(
    r"./saved_models/encoder_v3",
    custom_objects={"custom_loss": None},
)

autoencoder_model.trainable = False
model_input = autoencoder_model.layers[0].input
encoder_output = autoencoder_model.get_layer("encoder_output").output
encoder = keras.Model(inputs=[model_input], outputs=[encoder_output])

x = x_in = keras.layers.Input((256,256,3))

encoder_output = encoder(x)


x = keras.layers.Flatten()(encoder_output)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(5, "softmax")(x)

# create model
classifier = keras.Model(inputs=[x_in], outputs=[x])

classifier.summary()

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


2024-01-04 21:10:32.595105: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-04 21:10:32.628278: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2024-01-04 21:10:32.628298: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-01-04 21:10:32.629019: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 model (Functional)          (None, 16, 16, 16)        6840      
                                                                 
 flatten (Flatten)           (None, 4096)              0         
                                                                 
 dropout (Dropout)           (None, 4096)              0         
                                                                 
 dense (Dense)               (None, 5)                 20485     
                                                                 
Total params: 27,325
Trainable params: 20,485
Non-trainable params: 6,840
_________________________________________________________________


# load classifier

In [3]:
classifier = tf.keras.models.load_model(
    r"./saved_models/classifier_v2"
)

classifier.summary( expand_nested=True)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 model_2 (Functional)        (None, 16, 16, 16)        6840      
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_layer (InputLayer)  [(None, 256, 256, 3)]     0         |
|                                                               |
| depthwise_conv2d (Depthwise  (None, 16, 16, 24)     6168      |
| Conv2D)                                                       |
|                                                               |
| conv2d (Conv2D)           (None, 16, 16, 16)        400       |
|                                                               |
| encoder_output (Conv2D)   (None, 16, 16, 16)        272       |
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯

In [4]:
for l in classifier.layers:
  if isinstance(l, tf.keras.Model):
    for l2 in l.layers:
      print(l2)

  print(l.name)

input_1
model_2
flatten
dropout
dense


In [6]:
denseLayer = classifier.get_layer(name="dense")

weights, bias = denseLayer.get_weights()



[[ 0.04483382 -0.1569699   0.09172858  0.01094788  0.00739748]
 [ 0.04682713 -0.0507858   0.0284922   0.04230333 -0.01297957]
 [-0.00579267  0.02524961  0.06513952  0.08179232 -0.08076644]
 ...
 [-0.01647455 -0.02641835 -0.01486348  0.00800985  0.03369443]
 [ 0.0591411  -0.02864591 -0.06526988 -0.11374501  0.11178447]
 [-0.08019418 -0.02005223 -0.02052966  0.02625673 -0.02770743]] [ 0.00114521 -0.00351018  0.00670802 -0.00961753  0.00172011]
